# Assignment 4: Data Mining for Networks

authors:

Lynda Attouche ~  Lenny Klump


---



#### Installing gym

In [1]:
!pip install cmake 'gym[atari]' scipy

#### Imports

In [2]:
import numpy as np
import random
import gym.spaces
env = gym.make("Taxi-v3")

## Escape Game

#### Defining states, actions, reward matrix and Qtable

In [3]:
states = 6 #number of states(rooms)
actions = 6 #number of actions (moving from a room to another)

#Defining the reward matrix
R = np.array([
              [-1.0,-1.0,-1.0,-1.0,0.0,-1.0],
              [-1.0,-1.0,-1.0,0.0,-1.0,100.0],
              [-1.0,-1.0,-1.0,0.0,-1.0,-1.0],
              [-1.0,0.0,0.0,-1.0,0.0,-1.0],
              [0.0,-1.0,-1.0,0.0,-1.0,100.0],
              [-1.0,0.0,-1.0,-1.0,0.0,100.0]
])

#Initializing the Qtable
qtable = np.zeros((states,actions))
print("Q-table dim: ", qtable.shape)

Q-table dim:  (6, 6)


#### Hyperparameters

In [4]:
epsilon = 1.0 #Max greed (100%)
epsilon_min = 0.005 #Min greed (0.05%)
epsilon_decay = 0.99993 #decay multiplied with eps each episode
episodes = 400 #Num of games
max_steps = 100 #Max steps per episode
learning_rate = 0.65 #Learning rate 
gamma = 0.8 #Discount factor

In [5]:
def q_learning(Q_table,epsilon, epsilon_min, epsilon_decay, episodes, max_steps, lr, gamma):
  """
    Updates Q-table
    @params:
            - Q_table (ndarray) : initialized q-table 
            - epsilon (float) : random number (to be used in epsilon-greedy search)
            - epsilon_min (float) : minimum value of epsilon
            - epsilon_decay (float) : used for the progressive decrease of epsilon
            - episodes (int): number of episodes of the game
            - max_steps (int) : maximum number of steps of an episode
            - lr (float) : learning rate
            - gamma (float) : discount factor
    @return:
            updated q-table (ndarray)
  """

  for episode in range(episodes):
    score = 0
    state = random.randint(0, 5)

    for _ in range(max_steps):

      invalid_move = True
      #Take best action in Q-table (exploitation)
      if np.random.uniform(0,1) > epsilon:
        action = np.argmax(Q_table[state,:])
      #Take random action (exploration)
      else:
        while invalid_move:
          rnd_action = random.randint(0,5)
          if R[state,:][rnd_action]!=-1: 
            invalid_move = False
            action = rnd_action

      #Take a step
      next_state = action
      reward = R[state,action]

      #add up score
      score += reward

      #Update Q-table 
      Q_table[state,action] = reward + gamma*np.max(Q_table[next_state,:])

      if state == 5: break

      #Update state
      state = next_state

    #Reducing epsilon each episode (exploration-exploitation trade-off)
    if epsilon >= epsilon_min: epsilon *= epsilon_decay


In [6]:
q_learning(qtable,epsilon, epsilon_min, epsilon_decay, episodes, max_steps, learning_rate, gamma)


In [7]:
for i in range(6):
  print(qtable[i])

[  0.   0.   0.   0. 400.   0.]
[  0.   0.   0. 320.   0. 500.]
[  0.   0.   0. 320.   0.   0.]
[  0. 400. 256.   0. 400.   0.]
[320.   0.   0. 320.   0. 500.]
[  0. 400.   0.   0. 400. 500.]


## Taxi game

#### Getting the state & action space and creating Q-Learning table

In [8]:
state_space = env.observation_space.n
action_space = env.action_space.n
qtable_ = np.zeros((state_space,action_space))
print("Q-table dim: ",qtable_.shape)

Q-table dim:  (500, 6)


#### Defining hyperparameters

In [9]:
#same as the previous hyperparameters except gamma and number of episodes
episodes_ = 50000 #Num of games
gamma_ = 0.65 #Discount factor

#### Defining Q_Learning algorithm

In [10]:
def q_learning(qtable,epsilon, epsilon_min, epsilon_decay, episodes, max_steps, lr, gamma):
    """
    Updates Q-table
    @params:
            - Q_table (ndarray) : initialized q-table 
            - epsilon (float) : random number (to be used in epsilon-greedy search)
            - epsilon_min (float) : minimum value of epsilon
            - epsilon_decay (float) : used for the progressive decrease of epsilon
            - episodes (int): number of episodes of the game
            - max_steps (int) : maximum number of steps of an episode
            - lr (float) : learning rate
            - gamma (float) : discount factor
    @return:
            updated q-table (ndarray)
  """

    for episode in range(episodes):
        #Reset the game parameters
        state = env.reset() #current state
        done = False #to stop game
        score = 0
        for _ in range(max_steps):
            #Take best action in Q-table 
            #exploitation:
            if np.random.uniform(0,1) > epsilon:
                action = np.argmax(qtable[state,:])
            #Take random action 
            #exploration:
            else:
                action = env.action_space.sample()

            #stem the game forward
            next_state, reward, done, _ = env.step(action)

            #add up score
            score += reward

            #Update Q-table with new Qval
            qtable[state,action] = (1-learning_rate)*qtable[state,action] + learning_rate*(reward+gamma*np.max(qtable[next_state,:]))

            #Update state
            state = next_state

            if done:
              break

        #Reducing epsilon each episode (exploration-exploitation trade-off)
        if epsilon >= epsilon_min:
             epsilon *= epsilon_decay

In [11]:
q_learning(qtable_, epsilon, epsilon_min, epsilon_decay, episodes_, max_steps, learning_rate, gamma_)

In [12]:
qtable_

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ -2.65712496,  -2.54942301,  -2.65712496,  -2.54942301,
         -2.38372771, -11.54942301],
       [ -1.73663363,  -1.1332825 ,  -1.73663363,  -1.1332825 ,
         -0.20505   , -10.1332825 ],
       ...,
       [ -0.20505   ,   1.223     ,  -0.20505   ,  -1.1332825 ,
         -9.20505   ,  -9.20505   ],
       [ -2.38372771,  -2.12881186,  -2.38372771,  -2.12881186,
        -11.38372771, -11.38372771],
       [  6.8       ,   3.42      ,   6.8       ,  12.        ,
         -2.2       ,  -2.2       ]])

#### Comparing with a random agent


Concerning the random agent, it doesn't consider the state of our environment to select actions. They are selected randomly, that is why we call it random agent. 


We compare the Q-learning agent and the random agent according to the number of penaties and timesteps per episode and the number of rewards per action. 

In [13]:
def eval_qlAgent(qtable,episodes, max_steps):
  """
    Evaluates Q-learning agent
    @params:
            - Q_table (ndarray) : initialized q-table 
            - episodes (int): number of episodes of the game
            - max_steps (int) : maximum number of steps of an episode
    @return:
            - average number of timesteps (float) , average number of penalties (float)
  """


  total_epochs, total_penalties = 0, 0 #contrains respectively the number of epochs and penalties at the end

  for episode in range(episodes):
    #Reset the game parameters
    state = env.reset()
    done = False
    score = 0
    epochs, penalties = 0, 0 #contrains respectively the number of epochs and penalties for each episode

    for _ in range(max_steps):
      
      #we select the best action from the Q-table 
      action = np.argmax(qtable[state,:])

      #Take a step
      next_state, reward, done, _ = env.step(action)
      
      if reward == -10:
        penalties+=1

      #add up score
      score += reward

      #Update state
      state = next_state

      epochs+=1


      if done:
        break
    
    total_penalties += penalties
    total_epochs += epochs
  
  return total_epochs / episodes, total_penalties / episodes 

In [14]:
def eval_randomAgent(episodes, max_steps):
  """
    Evaluates Random Agent
    @params:
            - episodes (int): number of episodes of the game
            - max_steps (int) : maximum number of steps of an episode
    @return:
            - average number of timesteps (float) , average number of penalties (float)
  """

  
  total_epochs, total_penalties = 0, 0 #contrains respectively the number of epochs and penalties at the end
  
  for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0
    epochs, penalties = 0, 0 #contrains respectively the number of epochs and penalties for each episode

    for _ in range(max_steps):
      #As we said previously the agent select a random action as follows:
      action = env.action_space.sample()

      #Take a step
      next_state, reward, done, _ = env.step(action)
      
      
      if reward == -10:
        penalties+=1


      #add up score
      score += reward

      #Update state
      state = next_state

      epochs+=1


      if done:
        break
        
    total_penalties += penalties
    total_epochs += epochs
  
  return total_epochs / episodes, total_penalties / episodes 

In [15]:
t1,p1 = eval_qlAgent(qtable_,episodes_, max_steps)
t2,p2 = eval_randomAgent(episodes, max_steps)

print(f"Results after {episodes_} episodes:")
print("")
print("Average timesteps per episode:")
print("     Q-learning agent:",t1)
print("     Random agent:",t2)
print("")
print("Average penalties per episode:")
print("     Q-learning agent:",p1)
print("     Random agent:",p2)

Results after 50000 episodes:

Average timesteps per episode:
     Q-learning agent: 13.07462
     Random agent: 99.82

Average penalties per episode:
     Q-learning agent: 0.0
     Random agent: 32.2925


From these results, it can be seen how much more effective the Q-learning agent is than the random agent. Indeed, we have that:

*   The random agent performs a greater number of times than the Q-learning agent. This corresponds to a percentage equivalent to 761.8%

*   The Q-learning agent did not commit any penalty, which is the ideal and shows its performance. On the other hand, the random agent committed a fair number of penalties.
